In [1]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import statistics

import pandas as pd


Using TensorFlow backend.
/Users/yousif/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yousif/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yousif/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yousif/anaconda3/lib/python3.7/site-packages/t

# Univariate

In [2]:
def split_dataset(data, size, train_size):
    train_end = int(size * train_size)
    train, test = data[0:train_end], data[train_end:size]
    train = array(split(train, len(train)/4))
    test = array(split(test, len(test)/4))
    return train, test

# summarize scores
def summarize_scores(name, score,scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=4): 
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)

def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 0, 70, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # define model
    model = Sequential()
    model.add(Conv1D(32, 1, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

def forecast(model, history, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    # create forecast 
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

def evaluate_model(train, test, n_input):
    # fit model
    model = build_model(train, n_input)
    history = [x for x in train]
    # walk-forward validation 
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_sequence = forecast(model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next
        history.append(test[i, :])
    # evaluate predictions 
    predictions = array(predictions)
    score,scores = evaluate_forecasts(test[:, :, 0], predictions)  
    return score,scores, predictions, test[:, :, 0]


def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score
    for i in range(actual.shape[1]):
        # calculate rmse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        rmse = sqrt(mse)
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

In [3]:
def univariate_CNN(room_data, size, train_size, n_input):
    dataset = room_data
    train, test = split_dataset(dataset.values, size, train_size)
    score_list = list()
    scores_list = list()
    score,scores, predictions, test_set = evaluate_model(train, test, n_input)
    score_list.append(score)
    scores_list.append(scores)
    summarize_scores('cnn', score,scores)
    return predictions, test_set, score_list, scores_list

In [4]:
df = pd.read_csv('data/Run example/MDB 003.csv')
predictions, test_set, score_list, scores_list = univariate_CNN(df, 5000,.8, 4)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
cnn: [14.717] 9.4, 12.1, 15.7, 19.7


# Multivariate

In [5]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# split dataset into train/test sets
def split_dataset(data, size, train_size):
    train_end = int(size * train_size)
    train, test = data[0:train_end], data[train_end:size]
    train = array(split(train, len(train)/4))
    test = array(split(test, len(test)/4))
    return train, test

def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE
    for i in range(actual.shape[1]):
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        rmse = sqrt(mse)
        scores.append(rmse)
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

def summarize_scores(name, score,scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=4):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure enough data
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)

def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 0, 70, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # define model
    model = Sequential()
    model.add(Conv1D(16, 1, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast 
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_multivariate_cnn(train, test, n_input):
    # fit model
    model = build_model(train, n_input)
    history = [x for x in train]
    # walk-forward validation
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_sequence = forecast(model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting
        history.append(test[i, :])
    # evaluate predictions
    predictions = array(predictions)
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score,scores, predictions, test[:, :, 0]



In [6]:
def multivariate_CNN(room_data, size, train_size, n_input):
    dataset = room_data
    train, test = split_dataset(dataset.values, size, train_size)
    score_list = list()
    scores_list = list()

    score,scores, predictions, test_set = evaluate_multivariate_cnn(train, test, n_input)
    score_list.append(score)
    scores_list.append(scores)

    summarize_scores('CNN', score,scores)
    return predictions, test_set, score_list, scores_list


In [7]:
df = pd.read_csv('data/Run example/MDB 003.csv')
predictions, test_set, score_list, scores_list = multivariate_CNN(df, 5000,.8, 4)


CNN: [13.860] 8.3, 11.5, 15.0, 18.4
